In [2]:
# useing this variable for saving models and summaries
exp_name = "CNN_paper"

## Importing Libraries

In [3]:
%load_ext autoreload
%autoreload 2
import tensorflow as tf
import numpy as np
from hdf5storage import savemat
import pickle 

import sys
sys.path.append('../../src')

In [4]:
import models.CNN_model as CNN_model
import training
import evaluation.nn_eval as nn_eval

## Loading synthetic data

In [4]:
#loading data
data_path= '/data2/data/zebrahim/synthetic_dataset/data_8192_1000_3_4_4_.03_.03_.2' + 'processed_data.p'

with open(data_path, 'r') as fin:
    data = pickle.load(fin)
    
train_data = data['train_data']
validation_data = data['validation_data']
test_data = data['test_data']

train_gt = np.expand_dims(data['train_gt'], axis=-1)
validation_gt = np.expand_dims(data['validation_gt'], axis=-1)
test_gt = np.expand_dims(data['test_gt'], axis=-1)

index_of_changes = data['index_of_changes']
 

In [5]:
#index of changes for test dataset
gt_test_idx_changes = index_of_changes[900:1000]

## Model

In [6]:
# Model parameters
n_variables = 12
learning_rate = 0.001
batch_size = 10

convolution_levels = 5    #number of convolution to have in the network
pooling_stride = [2, 2, 2, 2, 2]
weights_shape = [[5, n_variables, 32],
                 [5, 32, 64],
                 [5, 64, 64],
                 [5, 64, 64],
                 [5, 64, 64]]
bias_shape = [32, 64, 64, 64, 64]
activation = tf.nn.relu
n_classes = 1

##positive weight for weighted cross entropy
s = train_data.shape
resolution = np.prod(pooling_stride)
gt_effective = np.reshape(train_gt, (s[0]*(s[1]/resolution), resolution))
pos_weight = (1.0 - gt_effective.max(axis=1).mean()) / gt_effective.max(axis=1).mean()

variables_weights = np.ones((n_classes,))

In [7]:
print pos_weight

3.18601822124


In [8]:
# Launch the graph
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)

In [9]:
summary_path = '../../summary/' + exp_name 
checkpoint_path = '../../model/' + exp_name + '/' + exp_name 

In [10]:
model = CNN_model.Model(n_variables = n_variables,
                        learning_rate = learning_rate,
                        n_classes = n_classes)

In [11]:
model.build_conv_net(pos_weight = pos_weight,
                     learning_rate = learning_rate,
                     convolution_levels = convolution_levels, 
                     pooling_stride = pooling_stride,
                     weights_shape = weights_shape,
                     bias_shape = bias_shape,
                     n_classes = n_classes,
                     activation = activation,
                     trainable_conv = True,
                     trainable_last = True,
                     variables_weights = variables_weights)

In [12]:
trainer = training.Trainer(model = model,
                    data_train = train_data,
                    ground_truth_train = train_gt,
                    data_validation = validation_data,
                    ground_truth_validation = validation_gt,
                    sess = sess,
                    summary_path = summary_path,
                    batch_size = batch_size,
                    global_step = 0,
                    eval_type = 'change')

In [13]:
trainer.train(max_iter = 10000,
                train_eval_step = 100, 
                validation_eval_step = 100,
                display_step = 100)

Tarin Iter 0, auc= 0.216128, fmeasure_max= 0.409858, precision= 0.259402, recall= 0.975884
Train Random guess:  , auc= 0.246761fmeasure_max= 0.391263, precision= 0.243595, recall= 0.993569
Validation Iter 0, auc= 0.213219, fmeasure_max= 0.382166, precision= 0.237500, recall= 0.977702
Validation Random guess:  auc= 0.224421, fmeasure_max= 0.371101, precision= 0.227823, recall= 1.000000
Minibatch Loss= 1.074060
Tarin Iter 100, auc= 0.423371, fmeasure_max= 0.447568, precision= 0.330935, recall= 0.691152
Train Random guess:  , auc= 0.234697fmeasure_max= 0.380497, precision= 0.235132, recall= 0.996661
Validation Iter 100, auc= 0.522157, fmeasure_max= 0.476556, precision= 0.431755, recall= 0.531732
Validation Random guess:  auc= 0.226710, fmeasure_max= 0.371483, precision= 0.228291, recall= 0.996569
Minibatch Loss= 1.054915
Tarin Iter 200, auc= 0.751895, fmeasure_max= 0.699714, precision= 0.771008, recall= 0.640489
Train Random guess:  , auc= 0.231601fmeasure_max= 0.367190, precision= 0.2248

In [14]:
#Save the model
saver = tf.train.Saver()
saver.save(sess, checkpoint_path, global_step=trainer.global_step)

'../model/CNN1001/CNN1001-10000'

In [ ]:
#load the model
saver = tf.train.Saver()
saver.restore(sess, "../../models/model" + exp_name + "/model")

In [15]:
#This is the model evaluation on test dataset
probability_of_pos= nn_eval.evaluate_model (model, sess, test_data, test_gt)

Testing auc = 0.768599 F1_max = 0.710711 prec = 0.760694 rec = 0.666892
Test Random guess: auc= 0.231661 fmeasure_max= 0.375806 , precision= 0.231398 , recall= 0.999662


In [16]:
#shape of probability_change is (batch size, T, num_classes)
#since num_classes = 1 we squeeze that axis
probability_change = np.squeeze(probability_of_pos, axis = 2)

In [17]:
savemat('../../poc/poc'+exp_name+'.mat', {'poc':probability_change})